In [3]:
import tensorflow as tf

import numpy as np

from random import randint

import time

import re

import matplotlib.pyplot as plt

In [4]:
wordsList = np.load('./data/wordsList.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('./data/wordVectors.npy')
print ('Loaded the word vectors!')
print(len(wordsList))
print(wordVectors.shape)

Loaded the word list!
Loaded the word vectors!
400000
(400000, 50)


In [5]:
baseballIndex = wordsList.index('baseball')
wordVectors[baseballIndex]

array([-1.9327  ,  1.0421  , -0.78515 ,  0.91033 ,  0.22711 , -0.62158 ,
       -1.6493  ,  0.07686 , -0.5868  ,  0.058831,  0.35628 ,  0.68916 ,
       -0.50598 ,  0.70473 ,  1.2664  , -0.40031 , -0.020687,  0.80863 ,
       -0.90566 , -0.074054, -0.87675 , -0.6291  , -0.12685 ,  0.11524 ,
       -0.55685 , -1.6826  , -0.26291 ,  0.22632 ,  0.713   , -1.0828  ,
        2.1231  ,  0.49869 ,  0.066711, -0.48226 , -0.17897 ,  0.47699 ,
        0.16384 ,  0.16537 , -0.11506 , -0.15962 , -0.94926 , -0.42833 ,
       -0.59457 ,  1.3566  , -0.27506 ,  0.19918 , -0.36008 ,  0.55667 ,
       -0.70315 ,  0.17157 ], dtype=float32)

In [6]:
maxSeqLength = 10 #Maximum length of sentence
numDimensions = 300 #Dimensions for each word vector
firstSentence = np.zeros((maxSeqLength), dtype='int32')
firstSentence[0] = wordsList.index("i")
firstSentence[1] = wordsList.index("thought")
firstSentence[2] = wordsList.index("the")
firstSentence[3] = wordsList.index("movie")
firstSentence[4] = wordsList.index("was")
firstSentence[5] = wordsList.index("incredible")
firstSentence[6] = wordsList.index("and")
firstSentence[7] = wordsList.index("inspiring")
#firstSentence[8] and firstSentence[9] are going to be 0
print(firstSentence.shape)
print(firstSentence) #Shows the row index for each word

(10,)
[    41    804 201534   1005     15   7446      5  13767      0      0]


In [7]:
with tf.compat.v1.Session() as sess:
    print(tf.nn.embedding_lookup(wordVectors,firstSentence).eval().shape)

(10, 50)


In [8]:
def getTrainBatch():

    labels = []

    arr = np.zeros([batchSize, maxSeqLength])

    for i in range(batchSize):

        if (i % 2) == 0:

            num = randint(1, 11499)

            labels.append([1, 0])

        else:

            num = randint(13499, 24999)

            labels.append([0, 1])

        arr[i] = ids[num - 1:num]

    return arr, labels


def getTestBatch():

    labels = []

    arr = np.zeros([batchSize, maxSeqLength])

    for i in range(batchSize):

        num = randint(11499, 13499)

        if num <= 12499:

            labels.append([1, 0])

        else:

            labels.append([0, 1])

        arr[i] = ids[num - 1:num]

    return arr, labels

In [9]:

def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean) #平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev', stddev) #标准差
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

In [10]:
maxSeqLength = 250

batchSize = 24  # 批处理大小

numDimensions = 250

lstmUnits = 64  # LSTM的单元个数

numClasses = 2  # 分类类别

iterations = 50000  # 训练次数

wordsList = np.load(r'.\data\wordsList.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist()  #

wordsList = [word.decode('UTF-8') for word in wordsList]  # 不然会报word not in vocab错误

wordVectors = np.load(r'.\data\wordVectors.npy')
print('Loaded the word vectors!')
print(len(wordsList))
print(wordVectors.shape)

ids = np.load(r'.\data\idsMatrix.npy')

tf.reset_default_graph()

with tf.name_scope("input"):
    input_data = tf.placeholder(tf.int32, shape=[batchSize, maxSeqLength], name="x_input")  # 占位符，必不可少
    labels = tf.placeholder(tf.int32, shape=[batchSize, numClasses], name="y_input")

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]), dtype=tf.float32)

data = tf.nn.embedding_lookup(wordVectors, input_data)

data = tf.cast(data, tf.float32)  # 由于版本的问题，这一步必不可少，将x的数据格式转化成dtype，有的版本可以不写

'''LSTM网络的构建'''

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)

lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)

value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

with tf.name_scope("lstm"):
    with tf.name_scope("weight"):
        weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
        variable_summaries(weight)
    with tf.name_scope("bias"):
        bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
        variable_summaries(bias)
    value = tf.transpose(value, [1, 0, 2])  # value的输出为[batchsize,length,hidden_size]

    last = tf.gather(value, int(value.get_shape()[0]) - 1)
    with tf.name_scope("wx_plus_b"):
        prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
with tf.name_scope("accuracy"):
    accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))
    tf.summary.scalar('accuracy', accuracy)
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))  # 计算交叉熵
    tf.summary.scalar('loss', loss)
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)  # 随机梯度下降最小化loss

with tf.Session() as sess:
    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())

    merged=tf.summary.merge_all()#合并所有的summary data的获取函数，merge_all 可以将所有summary全部保存到磁盘，以便tensorboard显示。如果没有特殊要求，一般用这一句就可一显示训练时的各种信息了。
    writer=tf.summary.FileWriter("./tensorboard/test1/",sess.graph)

    for i in range(iterations):

        # Next Batch of reviews

        nextBatch, nextBatchLabels = getTrainBatch()

        sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
        rs=sess.run(merged,feed_dict={input_data: nextBatch, labels: nextBatchLabels})#运行所有合并所有的图，获取summary data函数节点和graph是独立的，调用的时候也需要运行session
        writer.add_summary(rs,i)
        if (i + 1) % 1000 == 0 and i != 0:

            loss_ = sess.run(loss, {input_data: nextBatch, labels: nextBatchLabels})

            accuracy_ = sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})

            print("iteration {}/{}...".format(i + 1, iterations),

                  "loss {}...".format(loss_),

                  "accuracy {}...".format(accuracy_))

        if (i + 1) % 10000 == 0 and i != 0:
            save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i + 1)
            print("saved to %s" % save_path)

Loaded the word list!
Loaded the word vectors!
400000
(400000, 50)
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.so

In [20]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))
iterations = 10
for i in range(iterations):
    nextBatch, nextBatchLabels = getTestBatch()
    print("Accuracy for this batch:", (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100)

INFO:tensorflow:Restoring parameters from models\pretrained_lstm.ckpt-50000
Accuracy for this batch: 95.83333134651184
Accuracy for this batch: 95.83333134651184
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 70.83333134651184
Accuracy for this batch: 83.33333134651184
Accuracy for this batch: 79.16666865348816
Accuracy for this batch: 75.0
Accuracy for this batch: 79.16666865348816
Accuracy for this batch: 100.0
Accuracy for this batch: 91.66666865348816
